In [1]:
import pennylane as qml
from pennylane import numpy as np
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt


from pennylane_snowflurry.API.client import MonarqClient
from pennylane_snowflurry.processing.steps import ReadoutErrorMitigation

from pennylane_snowflurry.processing.config import MonarqDefaultConfig, MonarqDefaultConfigNoBenchmark, NoPlaceNoRouteConfig, EmptyConfig


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
load_dotenv()
HOST_ENV = os.getenv("HOST")
USER_ENV = os.getenv("USER")
ACCESS_TOKEN_ENV = os.getenv("ACCESS_TOKEN")
REALM_ENV = os.getenv("REALM")

num_wires = [1] # for MonarQ
num_shots = 1000


my_client = MonarqClient(HOST_ENV , USER_ENV, ACCESS_TOKEN_ENV)

transpiler_config = NoPlaceNoRouteConfig()

transpiler_config.steps.append(ReadoutErrorMitigation())  # avec ou sans mitigation du readout

dev = qml.device(
    "monarq.default",
    client=my_client,
    wires=num_wires,
    shots=num_shots,
    processing_config = transpiler_config)
# dev = qml.device("default.qubit", wires = num_wires, shots = num_shots)


In [6]:
devsim = qml.device('default.qubit', wires =1, shots = num_shots)
@qml.qnode(dev, diff_method="parameter-shift")

def circuit(params):
    qml.RX(params[0], wires=1)
    return qml.counts(all_outcomes = True)
 #return qml.expval(qml.PauliZ(0)) # Equivalent to the operator mean value calculation presented in section 3, the mean value of operator <Z> = cos(theta)


def compute_expval(counts):
    val = list(counts.values())
    p0= val[0]/num_shots
    p1= val[1]/num_shots
    return p0 - p1


In [7]:
from datetime import datetime

def cost(params): 
    counts = circuit(params)
    return compute_expval(counts)

opt= qml.SPSAOptimizer(maxiter= 10000, c=0.02 ) 
params = np.array([5])
steps = 100

saved_params = []
saved_costs = []

start = datetime.now()
for i in range(steps):
    op = datetime.now()
    params = opt.step(cost, params)
    current_cost = cost(params)
    saved_params.append(params.copy())  
    saved_costs.append(current_cost)
  
    print(f'{i}. Optimized params: {params} ({(datetime.now() - op).total_seconds()}s) ({(datetime.now() - start).total_seconds()}s)') # cos(pi) = -1 !



0. Optimized params: [4.9475] (45.173918s) (45.174479s)
1. Optimized params: [4.97697648] (45.009639s) (90.184118s)
2. Optimized params: [4.84863554] (46.6994s) (136.883518s)
3. Optimized params: [4.7079784] (43.439074s) (180.322592s)
4. Optimized params: [4.5876752] (45.521136s) (225.843728s)
5. Optimized params: [4.40547013] (44.179611s) (270.023339s)
6. Optimized params: [4.32360551] (44.457757s) (314.481096s)
7. Optimized params: [4.29903467] (49.655135s) (364.136231s)


KeyboardInterrupt: 

In [8]:
for i, (param, cost_val) in enumerate(zip(saved_params, saved_costs)):
    print(f"Step {i + 1}: Params = {param}, Cost = {cost_val:.4f}")

Step 1: Params = [3.495], Cost = -0.9420
Step 2: Params = [3.47088288], Cost = -0.9380
Step 3: Params = [3.44298267], Cost = -0.9580
Step 4: Params = [3.42001824], Cost = -0.9740
Step 5: Params = [3.36426798], Cost = -0.9660
Step 6: Params = [3.37322888], Cost = -0.9800
Step 7: Params = [3.33381258], Cost = -0.9820
Step 8: Params = [3.30002767], Cost = -0.9860
Step 9: Params = [3.28760264], Cost = -0.9900
Step 10: Params = [3.29074026], Cost = -0.9760
Step 11: Params = [3.29390634], Cost = -0.9820
Step 12: Params = [3.28432996], Cost = -0.9940
Step 13: Params = [3.25216872], Cost = -0.9980
Step 14: Params = [3.23921528], Cost = -0.9940
Step 15: Params = [3.22943806], Cost = -0.9980
Step 16: Params = [3.21960274], Cost = -0.9980
Step 17: Params = [3.20971286], Cost = -0.9920
Step 18: Params = [3.20971286], Cost = -0.9980
Step 19: Params = [3.19639316], Cost = -0.9980
Step 20: Params = [3.19304791], Cost = -0.9980
Step 21: Params = [3.19304791], Cost = -1.0000
Step 22: Params = [3.193047

In [ ]:
plt.figure(figsize=(10, 6))

optimized_params = [p[0] for p in saved_params]
plt.plot(range(1, steps + 1), optimized_params, marker="o", linestyle="-", label="Optimized Parameter")
plt.axhline(np.pi, color="red", linestyle="--", label=" π")

plt.xlabel("Step")
plt.ylabel("Parameter Value")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import json

# Save to JSON file
results = [{"step": i + 1, "params": param.tolist(), "cost": cost_val} 
           for i, (param, cost_val) in enumerate(zip(saved_params, saved_costs))]

with open("optimization_results.json", "w") as f:
    json.dump(results, f, indent=4)